In [1]:
print("hello world")

hello world


In [2]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
import torch

In [4]:
device = 'cpu'

In [5]:
with open("input.txt","r",encoding="utf-8") as f:
    text = f.read()

In [6]:
print(text[:10])

First Citi


In [7]:
print(len(text))

1115394


In [8]:
chars = sorted(list(set(text)))

In [9]:
vocab_size = len(chars)

In [10]:
print(vocab_size)
print(''.join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [11]:
#create a mapping from characters to integers
stoi= { ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars) }
encode =  lambda s: [stoi[c] for c in s]# encoder: take a string, output a list of integers
decode =  lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [12]:
print(encode(" !$&"))

[1, 2, 3, 4]


In [13]:
print(decode(encode(" !$&")))

 !$&


In [14]:
import torch

In [15]:
data = torch.tensor(encode(text), dtype=torch.long)

In [16]:
print(data.shape,data.type)

torch.Size([1115394]) <built-in method type of Tensor object at 0x000001C55DA584B0>


In [17]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [18]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [19]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [20]:
x = train_data[:block_size]

In [21]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split) :
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y

In [22]:
xb,yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [24]:
class BigramModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self,inputs,targets = None):
        logits = self.token_embedding_table(inputs)
        if targets is None:
            loss =None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,inputs,max_new_tokens):
        for _ in range(max_new_tokens):
            logits , loss = self(inputs)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=1)
            inputs_next = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs,inputs_next),dim=1)
        return inputs

model = BigramModel(vocab_size)
m= model.to(device)
logits,loss = m(xb,yb) 

In [25]:
print(loss.item())

4.878634929656982


In [26]:
inputs = torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(inputs=inputs,max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [27]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [28]:
batch_size=32
for steps in range(10000):
    xb,yb = get_batch('train')
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.382369041442871


In [29]:

print(decode(m.generate(inputs=inputs,max_new_tokens=200)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo


In [33]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [34]:
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1,keepdim=True)
xbow = wei @ x

In [37]:
xbow[:1]

tensor([[[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
           2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
          -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
           1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
          -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
          -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
           1.5236e+00,  2.5086e+00],
         [-2.4116e-01, -1.6063e-01,  3.2526e-01, -3.9683e-01,  3.9208e-01,
           5.7976e-01, -9.9692e-02, -1.1702e-01, -7.3193e-02,  1.2198e-01,
          -4.0159e-01, -1.0025e+00, -4.8488e-01,  1.6602e-01, -7.5923e-01,
           4.2481e-01,  1.4972e+00, -4.7132e-01,  8.1860e-01,  3.4460e-01,
          -1.7740e+00,  1.2990e+00,  2.1250e+00, -5.7775e-01,  7.8881e-01,
          -1.5365e+00, -1.6947e-01, -2.7318e-01,  6.1334e-01,  3.6581e-01,
           1.5667e+00,  1.0527e+00],
         [-4.3893e-01,  9.